In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE33649"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE33649"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE33649.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE33649.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE33649.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Inter-ethnic differences in lymphocyte  sensitivity to glucocorticoids reflect variation in transcriptional response"
!Series_summary	"Glucocorticoids (GCs) are steroid hormones widely  used as pharmaceutical interventions, which act mainly by regulating gene expression levels. A large fraction of patients (~30%), especially those of African descent, show a weak response to treatment. To interrogate the contribution of variable transcriptional response to inter-ethnic differences, we measured  in vitro lymphocyte GC sensitivity (LGS) and transcriptome-wide response to GCs in peripheral blood mononuclear cells (PBMCs) from African-American and European-American healthy donors. We found that transcriptional response after 8hrs treatment was significantly correlated with variation in LGS within and  between populations. We found that NFKB1, a gene previously found to predict LGS within populations, was more strongly downregulated in European-American

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import re
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains mRNA expression data from PBMCs
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For Glucocorticoid_Sensitivity trait
# Key 3 contains "in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex)" with various values
trait_row = 3

# For age
# Key 6 contains "age (years)" with various values
age_row = 6

# For gender
# Key 5 contains "gender" with values female and male
gender_row = 5

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait data (GC sensitivity) to continuous value."""
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    if not isinstance(value, str):
        return None
    
    if 'in vitro lymphocyte gc sensitivity' not in value.lower():
        return None
    
    # Extract numeric value using regex
    match = re.search(r'(\d+\.\d+)', value)
    if match:
        return float(match.group(1))
    return None

def convert_age(value):
    """Convert age data to continuous value."""
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    if not isinstance(value, str):
        return None
    
    if 'age' not in value.lower():
        return None
    
    # Extract numeric value using regex
    match = re.search(r'(\d+\.\d+)', value)
    if match:
        return float(match.group(1))
    return None

def convert_gender(value):
    """Convert gender data to binary (0: female, 1: male)."""
    if value is None:
        return None
    if isinstance(value, (int, float)):
        return float(value)
    if not isinstance(value, str):
        return None
    
    if 'gender' not in value.lower():
        return None
    
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    return None

# 3. Save Metadata
# Trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Save the cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical feature extraction
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in previous output
    # Sample characteristics dictionary structure is {row_index: [values]}
    # We need to transform it into a DataFrame with columns being sample IDs
    
    # Example sample characteristics dictionary from previous output
    sample_chars = {
        0: ['cell type: peripheral blood mononuclear cells'],
        1: ['population: African-American', 'population: European-American'],
        2: ['treatment: dexamethasone', 'treatment: vehicle (EtOH)'],
        3: ['in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 89.43486', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 95.88507',
            'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 95.22036', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 92.86704',
            'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 93.71633', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 96.76962',
            'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 88.55031', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 90.09957',
            'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 94.17097', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 86.97089',
            'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 98.34904', 'in vitro lymphocyte gc sensitivity (lgs - %inhibition by dex): 91.14896'],
        4: ['duration of treatment (hours): 8', 'duration of treatment (hours): 24'],
        5: ['gender: female', 'gender: male'],
        6: ['age (years): 44.15342', 'age (years): 24.72329', 'age (years): 32.37808', 'age (years): 20.38082',
            'age (years): 21.2411', 'age (years): 22.54247', 'age (years): 26.13973', 'age (years): 21.5616',
            'age (years): 21.9863', 'age (years): 26.76712', 'age (years): 23.59452', 'age (years): 23.47945']
    }
    
    # Determine number of samples
    max_samples = max(len(values) for values in sample_chars.values())
    
    # Create a DataFrame where each column is a sample
    clinical_data = pd.DataFrame(index=sample_chars.keys(), columns=range(max_samples))
    
    # Fill in values
    for row_idx, values in sample_chars.items():
        for col_idx, value in enumerate(values):
            clinical_data.loc[row_idx, col_idx] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted data
    preview = preview_df(selected_clinical_df)
    print("Clinical data preview:", preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical data preview: {0: [89.43486, 44.15342, 0.0], 1: [95.88507, 24.72329, 1.0], 2: [95.22036, 32.37808, nan], 3: [92.86704, 20.38082, nan], 4: [93.71633, 21.2411, nan], 5: [96.76962, 22.54247, nan], 6: [88.55031, 26.13973, nan], 7: [90.09957, 21.5616, nan], 8: [94.17097, 21.9863, nan], 9: [86.97089, 26.76712, nan], 10: [98.34904, 23.59452, nan], 11: [91.14896, 23.47945, nan]}
Clinical data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE33649.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Found data marker at line 67
Header line: "ID_REF"	"GSM832137"	"GSM832138"	"GSM832139"	"GSM832140"	"GSM832141"	"GSM832142"	"GSM832143"	"GSM832144"	"GSM832145"	"GSM832146"	"GSM832147"	"GSM832148"	"GSM832149"	"GSM832150"	"GSM832151"	"GSM832152"	"GSM832153"	"GSM832154"	"GSM832155"	"GSM832156"	"GSM832157"	"GSM832158"	"GSM832159"	"GSM832160"	"GSM832161"	"GSM832162"	"GSM832163"	"GSM832164"	"GSM832165"	"GSM832166"	"GSM832167"	"GSM832168"	"GSM832169"	"GSM832170"	"GSM832171"	"GSM832172"	"GSM832173"	"GSM832174"	"GSM832175"	"GSM832176"	"GSM832177"	"GSM832178"	"GSM832179"	"GSM832180"	"GSM832181"	"GSM832182"	"GSM832183"	"GSM832184"
First data line: "ILMN_1343291"	14.12073024	14.1847953	14.3271103	14.21074679	14.35649097	14.21573196	14.25949372	14.26541254	14.36153392	14.25490712	14.28494604	14.21327393	14.37099787	14.37099787	14.32494472	14.32079848	14.26699913	14.08661628	14.33650015	14.33877929	14.24410318	14.21573196	14.34573164	14.38961689	14.32959504	14.31869455	14.37099787	14.4243792	14.31077

Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651199', 'ILMN_1651209',
       'ILMN_1651210', 'ILMN_1651221', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651235', 'ILMN_1651236',
       'ILMN_1651237', 'ILMN_1651238', 'ILMN_1651249', 'ILMN_1651253',
       'ILMN_1651254', 'ILMN_1651259', 'ILMN_1651260', 'ILMN_1651262'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Analyzing the gene identifiers from the provided output
# The identifiers follow the pattern "ILMN_xxxxxxx", which are Illumina probe IDs
# These are not direct human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


Examining SOFT file structure:
Line 0: ^DATABASE = GeoMiame
Line 1: !Database_name = Gene Expression Omnibus (GEO)
Line 2: !Database_institute = NCBI NLM NIH
Line 3: !Database_web_link = http://www.ncbi.nlm.nih.gov/geo
Line 4: !Database_email = geo@ncbi.nlm.nih.gov
Line 5: ^SERIES = GSE33649
Line 6: !Series_title = Inter-ethnic differences in lymphocyte  sensitivity to glucocorticoids reflect variation in transcriptional response
Line 7: !Series_geo_accession = GSE33649
Line 8: !Series_status = Public on Feb 01 2012
Line 9: !Series_submission_date = Nov 12 2011
Line 10: !Series_last_update_date = Aug 13 2018
Line 11: !Series_pubmed_id = 22158329
Line 12: !Series_summary = Glucocorticoids (GCs) are steroid hormones widely  used as pharmaceutical interventions, which act mainly by regulating gene expression levels. A large fraction of patients (~30%), especially those of African descent, show a weak response to treatment. To interrogate the contribution of variable transcriptional respon


Gene annotation preview:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180, 6510136, 7560739, 1450438, 1240647], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan, nan], 'SEQUENCE': ['GAATAAAGAACAATCTGCTGATGATCCCTCCGTGGATCTGATTCGTGTAA', 'CCATGTGATACGA

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the gene annotation dataframe
# Based on examining the annotation preview, we need:
# - "ID" column which contains the same Illumina probe IDs seen in the gene expression data (ILMN_*)
# - "Symbol" column which contains the gene symbols we want to map to

# 2. Extract the mapping between probe IDs and gene symbols
mapping_df = gene_annotation[['ID', 'Symbol']]
mapping_df = mapping_df.dropna()  # Remove rows with missing gene symbols
# Rename 'Symbol' to 'Gene' to match the expected column name in apply_gene_mapping function
mapping_df = mapping_df.rename(columns={'Symbol': 'Gene'})
print(f"Gene mapping dataframe shape: {mapping_df.shape}")
print("First few rows of the mapping dataframe:")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene-level expression
# This will handle the many-to-many relationship between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Resulting gene expression dataframe shape: {gene_data.shape}")
print("First few gene symbols and their expression values:")
print(gene_data.head())

# Normalize gene symbols to standard format
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization, gene expression dataframe shape: {gene_data.shape}")
print("First few normalized gene symbols:")
print(gene_data.index[:10])

# Save gene expression data to CSV
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping dataframe shape: (44837, 2)
First few rows of the mapping dataframe:
             ID                     Gene
0  ILMN_1343048      phage_lambda_genome
1  ILMN_1343049      phage_lambda_genome
2  ILMN_1343050  phage_lambda_genome:low
3  ILMN_1343052  phage_lambda_genome:low
4  ILMN_1343059                     thrB
Resulting gene expression dataframe shape: (21372, 48)
First few gene symbols and their expression values:
       GSM832137  GSM832138  GSM832139  GSM832140  GSM832141  GSM832142  \
Gene                                                                      
A1BG   15.956246  15.847209  15.781695  15.764754  15.795053  15.643423   
A1CF   23.307923  23.242826  23.307606  23.256966  23.446269  23.527114   
A26C3  23.488414  23.404135  23.448766  23.626364  23.472122  23.514723   
A2BP1  31.002495  30.914475  30.992848  30.959287  30.961103  30.967724   
A2LD1   9.334543   9.229682   9.447489   9.405594   8.231067   8.343261   

       GSM832143  GSM832144  GSM832145 

After normalization, gene expression dataframe shape: (20259, 48)


First few normalized gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT',
       'A4GNT', 'AAA1', 'AAAS'],
      dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE33649.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_data = pd.read_csv(out_clinical_data_file)
print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(clinical_data.head())

# 3. Link clinical and genetic data
# First, transpose gene expression data to have samples as rows
gene_data_t = normalized_gene_data.T

# Print sample IDs from both datasets to debug the mismatch
print("\nSample ID diagnostics:")
print(f"Gene data sample IDs: {list(gene_data_t.index)[:5]}")
print(f"Clinical data columns: {list(clinical_data.columns)}")

# The clinical data has the trait, age, and gender values for each sample
# Gene expression data uses GSM IDs for samples (GSM832137, etc.)
# We need to map the sample IDs properly

# Transform sample IDs in gene_data to match clinical data indices
# Extract actual trait and demographic columns from clinical data
clinical_columns = clinical_data.columns.tolist()
trait_col = clinical_columns[0]  # First column should be the trait
print(f"Identified trait column: {trait_col}")

# Prepare gene expression data to be merged with clinical data
gene_data_t = gene_data_t.reset_index().rename(columns={'index': 'Sample_ID'})

# Construct a mapping between GSM IDs and numeric indices
# Get the unique IDs from the matrix file header
with gzip.open(matrix_file, 'rt') as file:
    for line in file:
        if '!series_matrix_table_begin' in line:
            # Next line is the header with GSM IDs
            header_line = next(file).strip()
            header_items = header_line.split('\t')
            gsm_ids = [id.strip('"') for id in header_items[1:]]  # Skip first column which is ID_REF
            break

# Map between GSM IDs and clinical data indices (1-based)
sample_mapping = {}
for i, gsm_id in enumerate(gsm_ids[:clinical_data.shape[0]]):
    sample_mapping[gsm_id] = i + 1  # 1-based index for clinical data

print(f"Sample mapping (first 5): {dict(list(sample_mapping.items())[:5])}")

# Apply the mapping to gene_data_t
gene_data_t['numeric_id'] = gene_data_t['Sample_ID'].map(sample_mapping)
gene_data_t = gene_data_t.dropna(subset=['numeric_id'])  # Keep only mapped samples
gene_data_t['numeric_id'] = gene_data_t['numeric_id'].astype(int)

# Set numeric_id as index to prepare for merge
gene_data_t = gene_data_t.set_index('numeric_id')
gene_data_t = gene_data_t.drop(columns=['Sample_ID'])

# Create the linked data by combining clinical and gene expression data
# First, get clinical data with samples as rows
clinical_data_t = clinical_data.T
clinical_data_t.index = pd.to_numeric(clinical_data_t.index, errors='coerce')
clinical_data_t = clinical_data_t.dropna(subset=[0])  # Keep samples with trait value

print("\nClinical data transposed:")
print(clinical_data_t.head())
print("\nGene data with mapped IDs:")
print(gene_data_t.head())

# Merge the datasets based on the index (numeric_id)
linked_data = clinical_data_t.merge(gene_data_t, 
                                   left_index=True, 
                                   right_index=True,
                                   how='inner')

# Rename the clinical columns to descriptive names
linked_data = linked_data.rename(columns={0: trait, 1: 'Age', 2: 'Gender'})

print(f"\nLinked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
if 'Age' in cleaned_data.columns:
    note += "Age data is available. "
if 'Gender' in cleaned_data.columns:
    note += "Gender data is available. "

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")

Gene data shape after normalization: (20259, 48)
Sample gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']


Gene data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE33649.csv
Clinical data shape: (3, 12)
Clinical data preview:
          0         1         2         3         4         5         6  \
0  89.43486  95.88507  95.22036  92.86704  93.71633  96.76962  88.55031   
1  44.15342  24.72329  32.37808  20.38082  21.24110  22.54247  26.13973   
2   0.00000   1.00000       NaN       NaN       NaN       NaN       NaN   

          7         8         9        10        11  
0  90.09957  94.17097  86.97089  98.34904  91.14896  
1  21.56160  21.98630  26.76712  23.59452  23.47945  
2       NaN       NaN       NaN       NaN       NaN  

Sample ID diagnostics:
Gene data sample IDs: ['GSM832137', 'GSM832138', 'GSM832139', 'GSM832140', 'GSM832141']
Clinical data columns: ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11']
Identified trait column: 0
Sample mapping (first 5): {'GSM832137': 1, 'GSM832138': 2, 'GSM832139': 3}

Clinical data transposed:
          0

Data shape after handling missing values: (3, 20262)
Quartiles for 'Glucocorticoid_Sensitivity':
  25%: 94.0437
  50% (Median): 95.22036
  75%: 95.552715
Min: 92.86704
Max: 95.88507
The distribution of the feature 'Glucocorticoid_Sensitivity' in this dataset is fine.

Quartiles for 'Age':
  25%: 22.552055
  50% (Median): 24.72329
  75%: 28.550684999999998
Min: 20.38082
Max: 32.37808
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 3 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Glucocorticoid_Sensitivity/GSE33649.csv
